In [34]:
df = dd.read_sql_table("testdaten", 'sqlite:///Kundendaten.db', "index")
df=df.compute()
df_BCG_1= df.groupby(["Angebotenes Produkt"])["Gewinn", "Anzahl"].sum().reset_index()
df_BCG_1.head()

<ipython-input-34-1be79145e5c6>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_BCG_1= df.groupby(["Angebotenes Produkt"])["Gewinn", "Anzahl"].sum().reset_index()


,Angebotenes Produkt,Gewinn,Anzahl
0,Altersvorsorge,17906.07,1911
1,Bausparvertrag,26511.47,19073
2,Depotkonto,12712.48,5456
3,Girokonto,10430.80,26077
4,Kredit,33766.60,96476


In [35]:
df_BCG_2=df.groupby(["Angebotenes Produkt"])["Anzahl"].apply(lambda x: x.sum()/x.count()).reset_index()
df_BCG_2.head()

,Angebotenes Produkt,Anzahl
0,Altersvorsorge,0.319352
1,Bausparvertrag,0.490447
2,Depotkonto,0.304839
3,Girokonto,0.582141
4,Kredit,0.579888


In [36]:
df_BCG=df_BCG_1.merge(df_BCG_2, on="Angebotenes Produkt")
df_BCG=df_BCG.rename(columns={"Anzahl_x": "Anzahl", "Anzahl_y":"Kaufwahrscheinlichkeit"})
df_BCG["Rentabilität"]=df_BCG["Gewinn"]/df_BCG["Anzahl"]
df_BCG.head(n=10)

,Angebotenes Produkt,Gewinn,Anzahl,Kaufwahrscheinlichkeit,Rentabilität
0,Altersvorsorge,17906.07,1911,0.319352,9.37
1,Bausparvertrag,26511.47,19073,0.490447,1.39
2,Depotkonto,12712.48,5456,0.304839,2.33
3,Girokonto,10430.80,26077,0.582141,0.40
4,Kredit,33766.60,96476,0.579888,0.35
5,Tagesgeldkonto,3391.08,1468,0.097978,2.31
6,Versicherung,33018.80,3880,0.436642,8.51


In [37]:
print("Gewinn: "+str(df_BCG["Gewinn"].mean()))
print("Anzahl: "+str(df_BCG["Anzahl"].mean()))
print("Kaufwahrscheinlichkeit: "+str(df_BCG["Kaufwahrscheinlichkeit"].mean()))
print("Rentabilität: " +str(df_BCG["Rentabilität"].mean()))

Gewinn: 19676.757142846454
Anzahl: 22048.714285714286
Kaufwahrscheinlichkeit: 0.40161228364236823
Rentabilität: 3.522857142856895


In [136]:
prob_mean=df_BCG["Kaufwahrscheinlichkeit"].mean()
rent_mean=df_BCG["Rentabilität"].mean()
prob_max=df_BCG["Kaufwahrscheinlichkeit"].max()
rent_max=df_BCG["Rentabilität"].max()

In [137]:
def BCG(K,R):
    if K<prob_mean and R<rent_mean:
        return "Poor Dog"
    elif K>prob_mean and R<rent_mean:
        return "Cash Cow"
    elif K>prob_mean and R>rent_mean:
        return "Star"
    else:
        return "Questionmark"

In [138]:
df_BCG["BCG"]=df_BCG.apply(lambda x: BCG(K=x["Kaufwahrscheinlichkeit"], R=x["Rentabilität"]), axis=1)
df_BCG.head(n=10)

,Angebotenes Produkt,Gewinn,Anzahl,Kaufwahrscheinlichkeit,Rentabilität,BCG
0,Altersvorsorge,17906.07,1911,0.319352,9.37,Questionmark
1,Bausparvertrag,26511.47,19073,0.490447,1.39,Cash Cow
2,Depotkonto,12712.48,5456,0.304839,2.33,Poor Dog
3,Girokonto,10430.80,26077,0.582141,0.40,Cash Cow
4,Kredit,33766.60,96476,0.579888,0.35,Cash Cow
5,Tagesgeldkonto,3391.08,1468,0.097978,2.31,Poor Dog
6,Versicherung,33018.80,3880,0.436642,8.51,Star


In [173]:
fig = px.scatter(df_BCG, x=df_BCG["Kaufwahrscheinlichkeit"]/prob_max, y=df_BCG["Rentabilität"]/rent_max, color="Angebotenes Produkt",
                labels={
                     "x": "Skalierte Kaufwahrscheinlichkeit",
                     "y": "Skalierte Rentabilität"
                 },)




fig.add_trace(go.Scatter(
    x=[0.25, 0.25],
    y=[0.45, 0.45],
    text=["Poor Dogs"],
    mode="text",showlegend = False
))
fig.add_trace(go.Scatter(
    x=[0.25, 0.25],
    y=[0.96, 0.96],
    text=["Questionmarks"],
    mode="text",showlegend = False
))
fig.add_trace(go.Scatter(
    x=[0.75, 0.75],
    y=[0.45, 0.45],
    text=["Cash Cows"],
    mode="text",showlegend = False
))
fig.add_trace(go.Scatter(
    x=[0.75, 0.75],
    y=[0.96, 0.96],
    text=["Stars"],
    mode="text",showlegend = False
))

fig.update_xaxes(range=[-0.01, 1.01], showgrid=False)
fig.update_yaxes(range=[-0.01, 1.01], showgrid=False)

fig.add_shape(type="rect",
    x0=-0.01, y0=-0.01, x1=0.5, y1=0.5,
    line=dict(color="RoyalBlue"),
)
fig.add_shape(type="rect",
    x0=0.5, y0=-0.01, x1=1.01, y1=0.5,
    line=dict(color="RoyalBlue"),
)
fig.add_shape(type="rect",
    x0=-0.01, y0=0.5, x1=0.5, y1=1.01,
    line=dict(color="RoyalBlue"),
)
fig.add_shape(type="rect",
    x0=0.5, y0=0.5, x1=1.01, y1=1.01,
    line=dict(color="RoyalBlue"),
)
        
fig.update_traces(marker=dict(size=20,line=dict(width=2)))